<a href="https://colab.research.google.com/github/chandankr014/Deep_learning/blob/main/sentiment_analysis_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/AarohiSingla/Aarohi/master/yelp.csv")

In [3]:
df.head()

,Unnamed: 0,sentence,label,source
0,0,Wow... Loved this place.,1,yelp
1,1,Crust is not good.,0,yelp
2,2,Not tasty and the texture was just nasty.,0,yelp
3,3,Stopped by during the late May bank holiday of...,1,yelp
4,4,The selection on the menu was great and so wer...,1,yelp


In [4]:
df.shape

(1000, 4)

In [5]:
x = df['sentence']
y = df['label']

In [85]:
xtrn, xtst, ytrn, ytst = train_test_split(x, y, random_state=3, test_size=0.3)

In [86]:
xtrn.shape, xtst.shape, ytrn.shape, ytst.shape

((700,), (300,), (700,), (300,))

---

In [87]:
from keras.preprocessing.text import Tokenizer

In [88]:
tokenizer = Tokenizer(num_words=20, lower=True)

In [89]:
tokenizer.fit_on_texts(xtrn)

In [90]:
xtrn[170]

'We also ordered the spinach and avocado salad; the ingredients were sad and the dressing literally had zero taste.'

In [91]:
xtrn = tokenizer.texts_to_sequences(xtrn)
xtst = tokenizer.texts_to_sequences(xtst)

In [92]:
xtrn[170]

[19, 8, 4, 11, 15]

In [93]:
ytrn.head()

926    0
411    1
224    0
14     0
170    0
Name: label, dtype: int64

In [94]:
from keras.utils.data_utils import pad_sequences

In [100]:
n = 20
xtrn_pad = pad_sequences(xtrn, padding='post', maxlen=n)
xtst_pad = pad_sequences(xtst, padding='post', maxlen=n)

In [101]:
xtrn[:5]

[[10, 4, 3, 1, 6, 10], [2], [10, 4, 11], [3, 4, 3, 4, 4], [1, 2, 1, 2, 1]]

In [102]:
xtrn_pad[:5]

array([[10,  4,  3,  1,  6, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [10,  4, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 3,  4,  3,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 1,  2,  1,  2,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

In [104]:
xtrn[170]

[19, 8, 4, 11, 15]

In [105]:
from keras.utils import to_categorical

In [106]:
n_classes = 2
y_trn = to_categorical(ytrn, n_classes)
y_tst = to_categorical(ytst, n_classes)

In [109]:
y_trn.shape

(700, 2)

In [111]:
y_tst[:7]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

##RNN Model

In [ ]:
!pip install Sequencial

In [150]:
from keras import Sequential

In [151]:
from keras.layers import Dense, Dropout, Activation, SimpleRNN

In [152]:
from keras import optimizers

In [153]:
x_trn = np.array(xtrn_pad).reshape((xtrn_pad.shape[0], xtrn_pad.shape[1], 1))

In [154]:
x_tst = np.array(xtst_pad).reshape((xtst_pad.shape[0], xtst_pad.shape[1], 1))

In [155]:
x_trn.shape, x_tst.shape

((700, 20, 1), (300, 20, 1))

In [202]:
##building model
def vanilla_RNN():
  model = Sequential()
  model.add(SimpleRNN(50, input_shape=(20,1), return_sequences=False))
  model.add(Dense(n_classes))
  model.add(Activation('softmax'))
  model.summary()

  adam = optimizers.Adam(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  
  return model

In [205]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=vanilla_RNN, epochs=500, batch_size=20)
model.fit(x_trn, y_trn)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_9 (SimpleRNN)    (None, 50)                2600      
                                                                 
 dense_9 (Dense)             (None, 2)                 102       
                                                                 
 activation_9 (Activation)   (None, 2)                 0         
                                                                 
Total params: 2,702
Trainable params: 2,702
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/500
35/35 [==============================] - 1s 6ms/step - loss: 0.7050 - accuracy: 0.4871
Epoch 2/500
35/35 [==============================] - 0s 5ms/step - loss: 0.6952 - accuracy: 0.4900
Epoch 3/500
35/35 [==============================] - 0s 5ms/step - loss: 0.6919 - accuracy: 0.5329
Epoch 4/500
35/35 [==============================] - 0s 5ms/step - loss: 0.6913 - accuracy: 0.5429
Epoch 5/500
35/35 [==============================] - 0s 5ms/step - loss: 0.6841 - accuracy: 0.5286
Epoch 6/500
35/35 [==============================] - 0s 5ms/step - loss: 0.6816 - accuracy: 0.5529
Epoch 7/500
35/35 [==============================] - 0s 6ms/step - loss: 0.6810 - accuracy: 0.5457
Epoch 8/500
35/35 [==============================] - 0s 6ms/step - loss: 0.6826 - accuracy: 0.5586
Epoch 9/500
35/35 [==============================] - 0s 6ms/step - loss: 0.6793 - accuracy: 0.5557
Epoch 10/500
35/35 [==============================] - 0s 5ms/step - loss: 0.6783 - accuracy: 0.5857
Epoch 11/

In [206]:
from sklearn.metrics import accuracy_score, confusion_matrix
y_pred = model.predict(x_tst)
y_tst_ = np.argmax(y_tst, axis=1)
print(accuracy_score(y_pred, y_tst_))

10/10 [==============================] - 0s 4ms/step
0.58


##testing model on new input

In [220]:
a1 = ["you are good",
     "they were happy",
     "cake is tasty",
     "food was delicious", 
     "film was so boring and very bad scenery all throughout the movie", 
     "waiters are so slow in service",
     "audience were angry on team effor today"
     ]
a = ["food is very tasty"]
a = tokenizer.texts_to_sequences(a)
a = np.array(a, dtype='object')
a = pad_sequences(a, padding='post', maxlen=20)
a = a.reshape((a.shape[0], a.shape[1], 1))
print(a.shape)
prediction = model.predict(np.array(a))
print(prediction)

(1, 20, 1)
1/1 [==============================] - 0s 31ms/step
[0]


0 - negative, 
1 - positive

In [190]:
p = model.predict(x_tst[0:10])
p

1/1 [==============================] - 0s 23ms/step


array([0, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [192]:
xtst[0:10]

[[4, 2],
 [1, 1, 7],
 [8, 7, 19],
 [10, 7, 1],
 [16, 3],
 [8, 13, 7, 18],
 [10, 7, 14, 5, 5],
 [8, 9, 8, 5],
 [3, 4],
 [2, 7, 19]]